I am building a LLM model from scratch for Question and Answering and I am making use of Stanford Question Answering Dataset gotten from Kaggle public repository and also available on huggingface as well.

# Seting up Environment

In [1]:
!pip install transformers
!pip install datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

Here, I'm trying to install some dependences that I will be needing to for my LLM model

# Load and Preprocess Data

In [2]:
import pandas as pd
from datasets import load_dataset

# Load the SQuAD training and validation dataset
train_data = pd.read_json("/content/train-v1.1.json")
validation_data = pd.read_json("/content/dev-v1.1.json")


The dataset is in a .json formart, and here I'm trying to load the dataset using the pandas library by calling the read_json method

In [3]:
# Inspect the structure of the data
print(train_data.head())
print(validation_data.head())

                                                data  version
0  {'title': 'University_of_Notre_Dame', 'paragra...      1.1
1  {'title': 'Beyoncé', 'paragraphs': [{'context'...      1.1
2  {'title': 'Montana', 'paragraphs': [{'context'...      1.1
3  {'title': 'Genocide', 'paragraphs': [{'context...      1.1
4  {'title': 'Antibiotics', 'paragraphs': [{'cont...      1.1
                                                data  version
0  {'title': 'Super_Bowl_50', 'paragraphs': [{'co...      1.1
1  {'title': 'Warsaw', 'paragraphs': [{'context':...      1.1
2  {'title': 'Normans', 'paragraphs': [{'context'...      1.1
3  {'title': 'Nikola_Tesla', 'paragraphs': [{'con...      1.1
4  {'title': 'Computational_complexity_theory', '...      1.1


The first five rows of each of the train and test dataset are printed out to have an insight of what the dataset contains.

In [4]:
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_metric
import torch


I tried to define a function to prepocess and prepare the dataset for the next stage which is tokenization

In [7]:
def prepare_data_for_tokenizer(data):
    """
    Processes a DataFrame containing raw SQuAD data,
    organizing it to be compatible with the BERT tokenizer.
    """

    # Initialize lists to store processed data
    ids = []
    titles = []
    contexts = []
    questions = []
    answers = []

    # Iterate over each row in the DataFrame
    for _, row in data.iterrows():
        document = row['data']
        qa_title = document['title']
        paragraphs = document['paragraphs']

        # Extract context, questions, and answers from each paragraph
        for paragraph in paragraphs:
            context = paragraph['context']
            qas = paragraph['qas']

            for qa in qas:
                qa_id = qa['id']
                question = qa['question']
                answer = qa['answers'][0]

                formatted_answer = {
                    'answer_start': [answer['answer_start']],
                    'text': [answer['text']]
                }

                # Append extracted data to corresponding lists
                ids.append(qa_id)
                titles.append(qa_title)
                contexts.append(context)
                questions.append(question)
                answers.append(formatted_answer)

    # Create a DataFrame from the organized data
    cleaned_data = {
        'id': ids,
        'title': titles,
        'context': contexts,
        'question': questions,
        'answers': answers
    }

    return pd.DataFrame(cleaned_data)

# Prepare the data
train_cleaned = prepare_data_for_tokenizer(train_data)
validation_cleaned = prepare_data_for_tokenizer(validation_data)


In [8]:
# Display the cleaned data to verify the structure
print(train_cleaned.head())
print(validation_cleaned.head())


                         id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   
2  5733be284776f41900661180  University_of_Notre_Dame   
3  5733be284776f41900661181  University_of_Notre_Dame   
4  5733be284776f4190066117e  University_of_Notre_Dame   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   
2  Architecturally, the school has a Catholic cha...   
3  Architecturally, the school has a Catholic cha...   
4  Architecturally, the school has a Catholic cha...   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at Notre Dame?   
4  What sits on top of the Main Building